In [19]:
%reload_ext autoreload
%autoreload 2

In [46]:
import taxi_demand_predictor.config as config
from taxi_demand_predictor.feature_store_api import get_feature_group, get_feature_store, get_project

In [47]:
project = get_project()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/71842


In [48]:
feature_store = get_feature_store()
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/71842
Connected. Call `.close()` to terminate connection gracefully.


In [49]:
print(f'{config.FEATURE_GROUP_NAME=}')
print(f'{config.FEATURE_GROUP_VERSION=}')

config.FEATURE_GROUP_NAME='time_series_hourly_feature_group'
config.FEATURE_GROUP_VERSION=1


In [50]:
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all(),
    )
except:
    print('Feature view already exists. Skipping creation.')

feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION,
)

Feature view already exists. Skipping creation.


In [51]:
ts_data, _ = feature_view.training_data(
    description="Time-series hourly taxi rides",
)

2023-08-06 18:36:44,533 INFO: USE `ariio_featurestore`
2023-08-06 18:36:44,929 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `ariio_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '1970-01-01 12:16:40.000' AND `fg0`.`pickup_hour` < TIMESTAMP '2023-08-07 01:36:43.000'


In [52]:
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
302450,2022-01-01 00:00:00,0,1
2834140,2022-01-01 01:00:00,0,1
1411934,2022-01-01 02:00:00,0,1
2301844,2022-01-01 03:00:00,0,1
496639,2022-01-01 04:00:00,1,1
...,...,...,...
246604,2023-08-06 20:00:00,7,265
246842,2023-08-06 21:00:00,9,265
247044,2023-08-06 22:00:00,3,265
247388,2023-08-06 23:00:00,6,265


In [53]:
from taxi_demand_predictor.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data=ts_data,
    input_seq_len=24*28,
    step_size=23
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour']  = targets

print(f'{features_and_target.shape=}')

  0%|          | 0/265 [00:00<?, ?it/s]DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  1%|          | 2/265 [00:00<00:13, 19.63it/s]DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  2%|▏  

features_and_target.shape=(143100, 675)


In [54]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from taxi_demand_predictor.data_split import train_test_split

cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1))
print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date=cutoff_date,
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2023-07-09 00:00:00')
X_train.shape=(135150, 674)
y_train.shape=(135150,)
X_test.shape=(7950, 674)
y_test.shape=(7950,)


In [55]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from taxi_demand_predictor.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [56]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=4)

[I 2023-08-06 18:44:02,826] A new study created in memory with name: no-name-eb3c19c7-6d7b-4bca-ac6c-84a395264863
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


[LightGBM] [Warning] bagging_fraction is set=0.2944729983469634, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2944729983469634
[LightGBM] [Warning] feature_fraction is set=0.36876707711145906, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.36876707711145906


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


[LightGBM] [Warning] bagging_fraction is set=0.2944729983469634, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2944729983469634
[LightGBM] [Warning] feature_fraction is set=0.36876707711145906, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.36876707711145906


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
[I 2023-08-06 18:44:24,470] Trial 0 finished with value: 3.3770943828393376 and parameters: {'num_leaves': 246, 'feature_fraction': 0.36876707711145906, 'bagging_fraction': 0.2944729983469634, 'min_child_samples': 74}. Best is trial 0 with value: 3.3770943828393376.
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information. 

...

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
[I 2023-08-06 18:44:48,594] Trial 2 finished with value: 3.3138044688184713 and parameters: {'num_leaves': 150, 'feature_fraction': 0.681135332725337, 'bagging_fraction': 0.8095567356554092, 'min_child_samples': 45}. Best is trial 2 with value: 3.3138044688184713.
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (

[LightGBM] [Warning] bagging_fraction is set=0.7897994741986363, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7897994741986363
[LightGBM] [Warning] feature_fraction is set=0.43298187202655153, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.43298187202655153


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


[LightGBM] [Warning] bagging_fraction is set=0.7897994741986363, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7897994741986363
[LightGBM] [Warning] feature_fraction is set=0.43298187202655153, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.43298187202655153


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
[I 2023-08-06 18:45:05,158] Trial 3 finished with value: 3.375839048513609 and parameters: {'num_leaves': 141, 'feature_fraction': 0.43298187202655153, 'bagging_fraction': 0.7897994741986363, 'min_child_samples': 67}. Best is trial 2 with value: 3.3138044688184713.


In [57]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 150, 'feature_fraction': 0.681135332725337, 'bagging_fraction': 0.8095567356554092, 'min_child_samples': 45}


In [58]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


[LightGBM] [Warning] bagging_fraction is set=0.8095567356554092, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8095567356554092
[LightGBM] [Warning] feature_fraction is set=0.681135332725337, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.681135332725337


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x2a0523940>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.8095567356554092,
                               feature_fraction=0.681135332725337,
                               min_child_samples=45, num_leaves=150))])

In [59]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=4.5709


See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [60]:
import joblib
from taxi_demand_predictor.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['/Users/auser/work/aiml/mine/taxi_demand_predictor/taxi_demand_predictor/models/model.pkl']

In [44]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [61]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name='taxi_demand_predictor_next_hour',
    metrics={'test_mae': test_mae},
    description='Predicts the number of taxi pickups in the next hour using LightGBM with some hyper-parameter tuning',
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(MODELS_DIR / 'model.pkl')

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


Model created, explore it at https://c.app.hopsworks.ai:443/p/71842/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)